In [69]:

import tensorflow as tf
print(tf.__version__)

2.8.2


In [70]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# New Section

In [71]:
from keras.models import Model
from keras.layers import Input,LSTM, Dense
import numpy as np

In [72]:
batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 10000

data_path = '/content/drive/MyDrive/Colab Notebooks/fra.txt'

In [73]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()


In [74]:
with open(data_path, 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')

lines[0:10]

['Go.\tVa !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)',
 'Go.\tMarche.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)',
 'Go.\tBouge !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #9022935 (Micsmithel)',
 'Hi.\tSalut !\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #509819 (Aiji)',
 'Hi.\tSalut.\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #4320462 (gillux)',
 'Run!\tCours\u202f!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #906331 (sacredceltic)',
 'Run!\tCourez\u202f!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #906332 (sacredceltic)',
 'Run!\tPrenez vos jambes à vos cous !\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #2077449 (sacredceltic)',
 'Run!\tFile !\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #2077454 (sacredceltic)',
 'Run!\tFilez !\tCC-BY 2.0 (France) Attribution: tatoeba

In [75]:
for line in lines[: min(num_samples, len(lines) - 1)]:
  input_text, target_text,_= line.split('\t')
# We use "tab" as the "start sequence" character
#for the targets, and "\n" as "end sequence" character.
  target_text = '\t' + target_text + '\n'
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)

In [83]:
input_characters

{' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '5',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'é'}

In [86]:
#target_characters

In [91]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [92]:
print(' Number of samples: ', len(input_texts))
print('Number of unique input tokens: ' , num_encoder_tokens)
print('Number of unique output tokens: ', num_decoder_tokens)
print('Max sequence length for inputs: ' , max_encoder_seq_length)
print('Max sequence length for outputs: ' , max_decoder_seq_length)

 Number of samples:  10000
Number of unique input tokens:  71
Number of unique output tokens:  93
Max sequence length for inputs:  15
Max sequence length for outputs:  59


In [93]:
input_token_index = dict(
[(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
[(char, i) for i, char in enumerate(target_characters)])

In [94]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '5': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 ':': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'Q': 36,
 'R': 37,
 'S': 38,
 'T': 39,
 'U': 40,
 'V': 41,
 'W': 42,
 'Y': 43,
 'a': 44,
 'b': 45,
 'c': 46,
 'd': 47,
 'e': 48,
 'f': 49,
 'g': 50,
 'h': 51,
 'i': 52,
 'j': 53,
 'k': 54,
 'l': 55,
 'm': 56,
 'n': 57,
 'o': 58,
 'p': 59,
 'q': 60,
 'r': 61,
 's': 62,
 't': 63,
 'u': 64,
 'v': 65,
 'w': 66,
 'x': 67,
 'y': 68,
 'z': 69,
 'é': 70}

In [96]:
encoder_input_data = np.zeros(
(len(input_texts), max_encoder_seq_length, num_encoder_tokens),
dtype='float32')
decoder_input_data = np.zeros(
(len(input_texts), max_decoder_seq_length, num_decoder_tokens),
dtype='float32')
decoder_target_data = np.zeros(
(len(input_texts), max_decoder_seq_length, num_decoder_tokens),
dtype= 'float32')

In [112]:
input_texts[1:10]

['Go.', 'Go.', 'Hi.', 'Hi.', 'Run!', 'Run!', 'Run!', 'Run!', 'Run!']

In [109]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[char]] = 1.
  encoder_input_data[i, t + 1 :, input_token_index[ ' ' ]] = 1.
  for t, char in enumerate(target_text):
    #decoder_target_data is ahead of decoder_input_data by one timestep
    decoder_input_data[i, t, target_token_index[char]] = 1.
    if t > 0:
    #decoder_target_data will be ahead by one timestep
    # and will not include the start character.
      decoder_target_data[i, t - 1, target_token_index[char]] = 1.
  decoder_input_data[i, t + 1 :, target_token_index[' ']] = 1.
  decoder_target_data[i, t :, target_token_index[' ']] = 1.

In [116]:
encoder_input_data.shape

(10000, 15, 71)

In [117]:
decoder_input_data.shape

(10000, 59, 93)

In [118]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
#We discard encoder_outputs and only keep the states.
encoder_states = [state_h, state_c]

In [121]:
#Set up the decoder, using encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
#We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_1stm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_1stm(decoder_inputs,initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation= 'softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [123]:
# Define the model that will turn
# encoder_input_data` & decoder_input_data into decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
#Run training
model.compile(optimizer= 'rmsprop', loss= 'categorical_crossentropy',metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
batch_size=batch_size,epochs= epochs, validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 10s 21ms/step - loss: 1.1251 - accuracy: 0.7369 - val_loss: 1.0536 - val_accuracy: 0.7141
Epoch 2/100
125/125 [==============================] - 2s 12ms/step - loss: 0.8154 - accuracy: 0.7806 - val_loss: 0.8255 - val_accuracy: 0.7690
Epoch 3/100
125/125 [==============================] - 2s 13ms/step - loss: 0.6600 - accuracy: 0.8159 - val_loss: 0.7131 - val_accuracy: 0.7951
Epoch 4/100
125/125 [==============================] - 2s 13ms/step - loss: 0.5774 - accuracy: 0.8324 - val_loss: 0.6395 - val_accuracy: 0.8119
Epoch 5/100
125/125 [==============================] - 2s 13ms/step - loss: 0.5285 - accuracy: 0.8454 - val_loss: 0.5899 - val_accuracy: 0.8278
Epoch 6/100
125/125 [==============================] - 2s 13ms/step - loss: 0.4921 - accuracy: 0.8555 - val_loss: 0.5618 - val_accuracy: 0.8360
Epoch 7/100
125/125 [==============================] - 2s 13ms/step - loss: 0.4635 - accuracy: 0.8630 - val_loss: 0.5441 - val_accuracy